In [1]:
import re
from plumbum import local

In [22]:
def convert_h(path):
    "Takes a path in, converts all h* files to end in h"

    ppath = local.cwd / path
    files = ppath // '*.h*'
    for file in files:
        if file.suffix != '.h':
            file.move(file.with_suffix('.h'))

Convert all `.hh` files to `.h`

In [23]:
convert_h("include/goofit/")
convert_h("include/goofit/PDFs")
convert_h("include/goofit/rootstuff/")

Convert usages of includes to new locations, and standardize usage and `.hh`/`.h`

In [13]:
base = local.cwd / 'include'
files = base // '*/*.h'
files += base // 'goofit/*/*.h'
rel = [f - base for f in files]

In [2]:
src = [a for a in (local.cwd).walk() if a.suffix in ['.cu', '.cc', '.cpp', '.h'] ]

In [6]:
src = [s for s in src if 'MCBooster' not in str(s)]

In [38]:
for name in src:
    with name.open('r') as f:
        contents = f.read()
    for r in rel:
        before = r.parts[-1]
        after = str(r)
        if before in contents:
            contents = re.compile(r'["<]'+before+r'[h?][">]').sub('"'+after+'"',contents)
    with name.open('w') as f:
        f.write(contents)